In [42]:
import json
import requests
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import csv

In [6]:
url = requests.get('http://data.wnba.com/data/5s/v2015/json/mobile_teams/wnba/2018/players/10_player_info.json').json()

In [26]:
print("Number of players:", len(url['pls']['pl']))

Number of players: 276


In [44]:
### Convert JSON object to pandas dataframe

player_list = []
for i in np.arange(len(url['pls']['pl'])):
    #print(url['pls']['pl'][i]['pc'])
    co = url['pls']['pl'][i]['co']
    dob = url['pls']['pl'][i]['dob']
    dy = url['pls']['pl'][i]['dy']
    fa = url['pls']['pl'][i]['fa']
    fn = url['pls']['pl'][i]['fn']
    ht = url['pls']['pl'][i]['ht']
    la = url['pls']['pl'][i]['la']
    ln = url['pls']['pl'][i]['ln']
    num = url['pls']['pl'][i]['num']
    pc = url['pls']['pl'][i]['pc']
    pid = url['pls']['pl'][i]['pid']
    pos = url['pls']['pl'][i]['pos']
    s = url['pls']['pl'][i]['s']
    sn = url['pls']['pl'][i]['sn']
    ta = url['pls']['pl'][i]['ta']
    tc = url['pls']['pl'][i]['tc']
    tid = url['pls']['pl'][i]['tid']
    tn = url['pls']['pl'][i]['tn']
    ty = url['pls']['pl'][i]['ty']
    wt = url['pls']['pl'][i]['wt']
    y = url['pls']['pl'][i]['y']
    
    player_list.append([co, dob, dy, fa, fn, ht, la, ln, num, pc, pid, pos, s, sn, ta, tc, tid, tn, ty, wt, y])

df_players = pd.DataFrame(player_list, columns = ['co', 'dob', 'dy', 'fa', 'fn', 'ht', 'la', 'ln', 'num', 'pc', 'pid', 'pos', 's', 'sn',
                                    'ta', 'tc', 'tid', 'tn', 'ty', 'wt', 'y'])

In [47]:
df_players.head()

,co,dob,dy,fa,fn,ht,la,ln,num,pc,pid,pos,s,sn,ta,tc,tid,tn,ty,wt,y
0,,1992-11-06,0,N,Rebecca,6-2,Australia,Allen,9,rebecca_allen,204296,G,A,Australia,NYL,New York,1611661313,Liberty,From,162.0,3
1,,1995-03-20,2017,N,Lindsay,5-8,Notre Dame,Allen,12,lindsay_allen,1628263,G,I,Notre Dame,NYL,New York,1611661313,Liberty,College,144.0,1
2,,1993-06-11,2015,N,Brittany,5-9,California,Boyd,15,brittany_boyd,204327,G,A,California,NYL,New York,1611661313,Liberty,College,156.0,3
3,,1996-07-31,0,N,Tashia,6-1,Western Kentucky,Brown,,tashia_brown,1628883,F,I,Western Kentucky,NYL,New York,1611661313,Liberty,College,NaN,0
4,USA,1988-12-05,2010,N,Tina,6-4,Connecticut/USA,Charles,31,tina_charles,202250,C,A,Connecticut,NYL,New York,1611661313,Liberty,College,192.0,8


In [48]:
### Save pc (player id) to pull historical stats for each player
plr_id = list(map(lambda x: x.replace('_', '-'), df_players['pc']))



In [191]:
print("Number of players:", len(plr_id))

Number of players: 276


In [ ]:
start_time = time.time()
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
browser = webdriver.Chrome(chrome_options=chrome_options)
df_overall = pd.DataFrame()

for plr in plr_id[:]:
    browser.get('http://www.wnba.com/player/'+str(plr)+'/#/gamelogs')
    action = ActionChains(browser) # create ActionChains object
    #browser.find_elements_by_xpath('//select[@name="filter_year"]')
    contract = browser.find_element_by_xpath('//select[@name="filter_year"]')
    action.move_to_element(contract).perform()
    time.sleep(2)
    #contract = browser.find_element_by_xpath('//div[@class="profile__filter"]')
    try:
        contract.click()
    except:
        popup = browser.find_element_by_xpath('//a[@onclick="window.PulseInsightsObject.survey.closeButtonClickedEvent()"]')
        time.sleep(1)
        popup.click()
        time.sleep(2)
        action.move_to_element(contract).perform()
        time.sleep(2)
        contract.click()


    time.sleep(2)
    contract = browser.find_element_by_xpath('//option[@value="2017-18"]')
    try:
        contract.click()

    except:
        pass

    rows = browser.find_elements_by_xpath('//section[@id="gamelogs"]/section/wnba-stat-table/div/div[@class="stat-table__overflow"]/table/tbody/tr/td')
    dates = browser.find_elements_by_xpath('//section[@id="gamelogs"]/section/wnba-stat-table/div/div[@class="stat-table__overflow"]/table/tbody/tr/th')
    games = len(dates)
    print("number of games:", games)
    row_index = 1
    date_index = 0
    row_list1 = []
    row_list2 = []
    date_list = []

    for row in rows:
        row_list1.append(row.text)
        if row_index % 15 == 0:
            row_list2.append(row_list1)
            row_list1 = []
            date_index += 1
        row_index += 1

    for date in dates:
        date_list.append([plr, date.text])
        
    df_dates = pd.DataFrame(date_list, columns = ['player','date'])
    df_stats = pd.DataFrame(row_list2, columns = ['opp', 'result', 'min', 'fgm-a', '3pm-a', 'ftm-a', 'oreb', 'dreb', 'reb', 'ast',
                                  'stl', 'blk', 'to', 'pf', 'pts'])
    df_player = pd.concat([df_dates,df_stats], axis = 1)
    df = pd.concat([df, df_player], axis = 0)
    print("Player", plr, "completed:", time.time() - start_time)

number of games: 0
Player rebecca-allen completed: 20.999489068984985
number of games: 28
Player lindsay-allen completed: 44.593056201934814
number of games: 2
Player brittany-boyd completed: 62.130003213882446


In [189]:
df

,player,date,opp,result,min,fgm-a,3pm-a,ftm-a,oreb,dreb,reb,ast,stl,blk,to,pf,pts
0,rebecca-allen,"SEP 03, 2017",NYL @ DAL,W,9,0 - 1,0 - 1,0 - 0,0,1,1,0,0,1,0,2,0
1,rebecca-allen,"SEP 01, 2017",NYL vs. SAN,W,6,2 - 3,0 - 0,0 - 0,0,0,0,0,0,0,0,1,4
2,rebecca-allen,"AUG 27, 2017",NYL vs. CHI,W,22,4 - 9,1 - 3,0 - 0,0,1,1,2,3,0,1,0,9
3,rebecca-allen,"AUG 25, 2017",NYL vs. WAS,W,3,1 - 1,0 - 0,0 - 0,0,1,1,0,0,0,0,1,2
4,rebecca-allen,"AUG 23, 2017",NYL @ IND,W,10,0 - 1,0 - 0,0 - 0,0,0,0,0,0,0,0,0,0
5,rebecca-allen,"AUG 20, 2017",NYL vs. MIN,W,3,1 - 1,0 - 0,0 - 0,0,1,1,0,0,0,0,0,2
6,rebecca-allen,"AUG 18, 2017",NYL @ CON,W,12,1 - 2,1 - 2,0 - 0,0,4,4,2,0,0,1,2,3
7,rebecca-allen,"AUG 13, 2017",NYL vs. LAS,W,4,0 - 1,0 - 0,0 - 0,0,2,2,1,0,1,0,0,0
8,rebecca-allen,"AUG 11, 2017",NYL @ ATL,W,9,0 - 2,0 - 0,2 - 2,0,1,1,1,0,0,0,0,2
9,rebecca-allen,"AUG 08, 2017",NYL vs. IND,W,7,2 - 5,0 - 2,0 - 0,2,1,3,0,1,0,2,0,4
